# Lecture 3: Password Attack with Differential-Power-Analysis (Kocher et al. 1999)

In [ ]:
%load_ext autoreload
%autoreload 2

import os
import random

import numpy as np
import plotly.graph_objects as pgo

from securec.capture import capture

In [ ]:
data = capture(
    platform="elmo",
    number_of_traces=1000,
    number_of_samples=0,
    fromfile=os.path.abspath("sbox_lookup.c"),
    inputfunction=lambda _: [random.randint(0, 255) for _ in range(16)],
)

In [ ]:
fig = pgo.Figure()
for d in data:
    fig.add_trace(pgo.Scatter(y=d["trace"]))
fig.show()

In [ ]:
from lascar.tools.aes import sbox

def split(arr, condition):
    return arr[np.array(condition)], arr[~np.array(condition)]

In [ ]:
d1, d2 = split(data, [sbox[d["input"][0] ^ 0x00] & 0x01 == 0 for d in data])

fig = pgo.Figure()
for d in data:
    fig.add_trace(
        pgo.Scatter(
            y=np.abs(np.mean(d1["trace"], axis=0) - np.mean(d2["trace"], axis=0))
        )
    )
fig.show()

In [ ]:
def aes_sbox_dpa(
    traces,
    key_byte_index=0,
    selection_bit_index=0,
):
    diffs = []
    for guess in range(256):
        d1, d2 = split(
            data,
            [
                sbox[d["input"][key_byte_index] ^ guess] & (1 << selection_bit_index) == 0
                for d in data
            ],
        )
        diffs.append(
            (
                np.max(
                    np.abs(np.mean(d1["trace"], axis=0) - np.mean(d2["trace"], axis=0))
                ),
                guess,
            )
        )
    return sorted(diffs, reverse=True)


In [ ]:
aes_sbox_dpa(data, key_byte_index=1)